In [1]:
import numpy as np
import pandas as pd

In [2]:
from constants import cleaned_data_filepath
participants = ["107", "904", "905", "906"]
ys = [np.loadtxt("%s%s_labels.txt" % (cleaned_data_filepath, s)) for s in participants]
df = pd.read_csv("extracted.csv")

In [3]:
from bad_trials import get_bad_trials, transform_ybad_indices
from first_participants_map import map_first_participants

ybad = get_bad_trials(participants, ys)
ys = map_first_participants(ys, participants)
y = np.concatenate(ys)
ybad = transform_ybad_indices(ybad, ys)
y[ybad] = -1

In [4]:
df_cleaned = df[y != -1]

In [5]:
y_cleaned = y[y != -1]

In [12]:
y_cleaned += 1
y_cleaned[y_cleaned < 8] = 0
y_cleaned[y_cleaned >= 8] = 1

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(LinearSVC())

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df_cleaned, y_cleaned, test_size=0.3)

In [20]:
scaler.fit(X_train)
model.fit(scaler.transform(X_train), y_train)

BaggingClassifier(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                           fit_intercept=True,
                                           intercept_scaling=1,
                                           loss='squared_hinge', max_iter=1000,
                                           multi_class='ovr', penalty='l2',
                                           random_state=None, tol=0.0001,
                                           verbose=0),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=10, n_jobs=None,
                  oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

In [21]:
np.mean(model.predict(scaler.transform(X_test)) != y_test)

0.5079365079365079

In [23]:
np.mean(model.predict(scaler.transform(X_train)) != y_train)

0.0